In [41]:
import numpy as np

In [ ]:
import pandas as pd
image_df = pd.read_csv('data/service_annot.csv')

In [4]:
image_df["file_name"].tolist()


['582.jpg',
 '582.jpg',
 '582.jpg',
 '582.jpg',
 '582.jpg',
 '582.jpg',
 '1200.jpg',
 '1200.jpg',
 '1200.jpg',
 '1200.jpg',
 '1200.jpg',
 '1200.jpg',
 '448.jpg',
 '448.jpg',
 '448.jpg',
 '448.jpg',
 '976.jpg',
 '976.jpg',
 '976.jpg',
 '976.jpg',
 '976.jpg',
 '976.jpg',
 '666.jpg',
 '666.jpg',
 '666.jpg',
 '666.jpg',
 '666.jpg',
 '666.jpg',
 '666.jpg',
 '272.jpg',
 '272.jpg',
 '272.jpg',
 '272.jpg',
 '272.jpg',
 '272.jpg',
 '1060.jpg',
 '1060.jpg',
 '1060.jpg',
 '1060.jpg',
 '1060.jpg',
 '1060.jpg',
 '1060.jpg',
 '1060.jpg',
 '1060.jpg',
 '1060.jpg',
 '472.jpg',
 '472.jpg',
 '472.jpg',
 '205.jpg',
 '205.jpg',
 '205.jpg',
 '205.jpg',
 '1007.jpg',
 '1007.jpg',
 '1007.jpg',
 '1198.jpg',
 '1198.jpg',
 '1198.jpg',
 '1198.jpg',
 '1198.jpg',
 '1198.jpg',
 '863.jpg',
 '863.jpg',
 '863.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '983.jpg',
 '845.jpg',
 '845.jpg',
 '845.jpg',
 '845.jpg',
 '845.jpg',
 '1

In [8]:
a = image_df["bbox"].tolist()

In [17]:
a = a.apply(lambda x: eval(x))

AttributeError: 'list' object has no attribute 'apply'

In [18]:
final_bboxes = [eval(bbox) for bbox in a]

In [21]:
type(final_bboxes[0][0])

int

In [22]:
image_df["class"].tolist()

['teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'caries',
 'caries',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'caries',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'caries',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'caries',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'caries',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'teeth',
 'caries',
 'c

In [44]:
classes_list = image_df["class"].unique().tolist()
# classes_list = video_info_df["class_str"].value_counts().index.tolist()

In [45]:
n_classes = len(classes_list)

In [46]:
bboxes = image_df["bbox"].tolist()
final_bboxes = [eval(bbox) for bbox in bboxes]

In [39]:
# predicted_classes = image_df["class"].tolist()

In [47]:
# Gets the smallest value needed to add to the end of the classes list to get a square matrix
root_round = np.ceil(np.sqrt(len(classes_list)))

In [48]:
root_round

2.0

In [49]:
total_size = root_round ** 2
padding_value = int(total_size - n_classes)
classes_padded = np.pad(classes_list, (0, padding_value), mode='constant')

# The padded matrix containing all the classes inside a matrix
classes_matrix = np.reshape(classes_padded, (int(root_round), int(root_round)))

# Flip it for better looks
classes_matrix = np.flip(classes_matrix, axis=0)

In [50]:
classes_matrix

array([['0', '0'],
       ['teeth', 'caries']], dtype='<U6')

In [ ]:
from transformers import ViTFeatureExtractor, ViTModel
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, models, transforms

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from pytorch_metric_learning import losses

In [ ]:
label2idx = {0:'teeth', 1:'caries'}
model = 'google/vit-base-patch16-384'
root_dir = '../input/'
batch_size = 8
num_labels = 2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained(model)

In [ ]:
model = ViTModel.from_pretrained(model)
model.to(device)

In [ ]:
model.eval()

res = []
base_embs = {}

for label in ['teeth', 'caries']:
    img_path = [i for i in os.listdir(f'../input/cutted-images-teeth/cutted_images_{label}') if i.endswith('.jpg')]

    for file in tqdm(img_path):
        image = Image.open(f'../input/cutted-images-teeth/cutted_images_{label}/{file}').convert('RGB')
        image = feature_extractor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            res.append(model(**image).pooler_output.detach().cpu().numpy())
            
    mean_emb = np.mean(np.asarray(res), axis=0)[0]
    base_embs.update({label:mean_emb})
pd.DataFrame(base_embs).to_csv('base_file_teeth_v2_30ep.csv', index=False)